## Data Cleaning

Dataset- https://shorturl.at/DV4X6

### Goal

- Clean and prepare a reproducible dataframe ready for analysis and plotting

1️⃣ Load & Basic Checks

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv(r"E:\Python DA\Delhi Metro Final Project\delhi_metro_raw.csv")
df.info(), df.head(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   TripID              150000 non-null  int64  
 1   Date                150000 non-null  object 
 2   From_Station        150000 non-null  object 
 3   To_Station          150000 non-null  object 
 4   Distance_km         150000 non-null  float64
 5   Fare                150000 non-null  float64
 6   Cost_per_passenger  150000 non-null  float64
 7   Passengers          148500 non-null  float64
 8   Ticket_Type         148500 non-null  object 
 9   Remarks             123743 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 11.4+ MB


(None,
    TripID        Date         From_Station            To_Station  Distance_km  \
 0   59771  2022-05-08             Inderlok         Kashmere Gate        12.94   
 1   21363  2023-01-12           Model Town        Dilshad Garden         2.33   
 2  127325  2023-07-13     Kashmere Gate     Netaji Subhash Place         5.56   
 3  140510  2022-11-10        Chandni Chowk             Hauz Khas         4.02   
 4  144298  2022-11-06          Rajiv Chowk        Kalkaji Mandir         9.66   
 
      Fare  Cost_per_passenger  Passengers   Ticket_Type      Remarks  
 0   77.99               18.27        13.0    Smart Card  maintenance  
 1   35.89               83.71        15.0  Tourist Card  maintenance  
 2   64.35               43.70         9.0        Single     off-peak  
 3  144.13               14.98        27.0           NaN  maintenance  
 4  104.96               83.84        23.0        Single     off-peak  ,
               TripID    Distance_km           Fare  Cost_per_pass

### Columns in the dataset

- `TripID`, `Date`, `From_Station`, `To_Station`, `Distance_km`, `Fare`, `Cost_per_passenger`, `Passengers`, `Ticket_Type`, `Remarks`

4️⃣ Missing Values Handling

Note: our dataset contains:
- ~1% missing values in `Passengers` (NaN)
- ~1% 'NA' strings in `Ticket_Type`
- ~1% blank `Remarks` entries
- ~30% `From_Station` values with leading/trailing spaces and inconsistent casing (needs trimming)

In [5]:
# To find the count of unique Ticket_Type

print(df["Ticket_Type"].value_counts())

Ticket_Type
Single          37341
Smart Card      37244
Return          37103
Tourist Card    36812
Name: count, dtype: int64


In [ ]:
#The first line finds all rows where the index is below 90000 and ticket type is “return”.
#The second line replaces the ticket type for those rows with “Tourist Card”

mask = (df.index < 90000) & (df["Ticket_Type"].astype(str).str.strip().str.lower() == "return")
df.loc[mask, "Ticket_Type"] = "Tourist Card"

In [7]:
print(df["Ticket_Type"].value_counts())

Ticket_Type
Tourist Card    59193
Single          37341
Smart Card      37244
Return          14722
Name: count, dtype: int64


Undertand Frequency of data in column -"Ticket Type"

In [8]:
print('\nTicket_Type value counts (including NA):')

print(df['Ticket_Type'].value_counts(dropna=False).head())


Ticket_Type value counts (including NA):
Ticket_Type
Tourist Card    59193
Single          37341
Smart Card      37244
Return          14722
NaN              1500
Name: count, dtype: int64


Find the no of blanks count in Remarks Column and count the null values in Passengers column

In [9]:
print('\nRemarks blanks count: ', (df['Remarks']== '').sum())

print('\nPassengers null count: ', df['Passengers'].isna().sum())


Remarks blanks count:  0

Passengers null count:  1500


### 1) Trim whitespace and fix casing in `From_Station`

In [10]:
print('Before sample:')
print(df['From_Station'].sample(8))

Before sample:
149456      Shivaji Park
86536        Rajiv Chowk
105721    Dilshad Garden
44477       Punjabi Bagh
82088          Old Delhi
111540       Kirti Nagar
61449         Model Town
1519        Shivaji Park
Name: From_Station, dtype: object


### Next step

In [11]:
df['From_Station'] = df['From_Station'].astype(str).str.strip().str.title()

| Part                 | Meaning                                                                             |
| -------------------- | ----------------------------------------------------------------------------------- |
| `df['From_Station']` | Selecting the column                                                                |
| `astype(str)`        | Convert data to string (in case any value is non-string)                            |
| `str.strip()`        | Remove **leading & trailing whitespace** (e.g. `" Rajiv Chowk "` → `"Rajiv Chowk"`) |
| `str.title()`        | Capitalize **first letter of each word** (e.g. `"rajiv chowk"` → `"Rajiv Chowk"`)   |

### Next Step

In [12]:
print('\nAfter sample:')
print(df['From_Station'].sample(8))


After sample:
103722                   Aiims
48426                Hauz Khas
140931               New Delhi
67752             Jasola Vihar
93726              Mandi House
32474     Netaji Subhash Place
59341              Laxmi Nagar
103175               Old Delhi
Name: From_Station, dtype: object


### 2) Handle 'NA' strings in `Ticket_Type` and missing entries

| Problem                                            | Example        |
| -------------------------------------------------- | -------------- |
| Dataset has `'NA'` as text, not real missing value | `'NA'` ≠ `NaN` |
| We want real missing values                        | `'NA'` → `NaN` |


In [13]:
# Replace 'NA' string with actual NaN and then inspect
df['Ticket_Type'] = df['Ticket_Type'].replace('NA', np.nan)

print('Ticket_Type nulls:', df['Ticket_Type'].isna().sum()) 

Ticket_Type nulls: 1500


## Next Step

Mean -> Average

Median -> Middle value

Mode -> Highest Frequency

In [15]:
#The first line finds the most common Ticket_Type value in the DataFrame, ignoring blank values.
#The second line prints that most frequent value.

mode_ticket = df['Ticket_Type'].mode(dropna=True)[0]
print('Mode ticket type:', mode_ticket)

Mode ticket type: Tourist Card


### 3) Convert blank `Remarks` to NaN

In [16]:
df['Remarks'] = df['Remarks'].replace('', np.nan) 
print('Remarks nulls after conversion:', df['Remarks'].isna().sum())

Remarks nulls after conversion: 26257


### 4) Handle missing `Passengers` values

Options:
- Impute with median or mean
- Drop rows (if only few)

In [17]:
# Example: impute with median passengers
median_pass = int(df['Passengers'].median(skipna=True))
print('Median passengers:', median_pass)

Median passengers: 20


## Next Step

In [18]:
df['Passengers'] = df['Passengers'].fillna(median_pass)

print('Passengers nulls after replacing:', df['Passengers'].isna().sum())

Passengers nulls after replacing: 0


### 5) Create `Profit` column

Profit per trip = (Fare - Cost_per_passenger) * Passengers

We'll create a `Profit` column and also a `Profit_per_passenger` for analysis.

In [19]:
df['Profit_per_passenger'] = df['Fare'] - df['Cost_per_passenger']

# Profit could be negative for subsidized trips -> keep as is
df['Profit'] = df['Profit_per_passenger'] * df['Passengers']



In [20]:
#lets print individual column to see it clearly
print(df[['Fare','Cost_per_passenger','Passengers','Profit_per_passenger','Profit']].head())

# lets also check total profit 
print('\nTotal profit across dataset:', df['Profit'].sum())

     Fare  Cost_per_passenger  Passengers  Profit_per_passenger   Profit
0   77.99               18.27        13.0                 59.72   776.36
1   35.89               83.71        15.0                -47.82  -717.30
2   64.35               43.70         9.0                 20.65   185.85
3  144.13               14.98        27.0                129.15  3487.05
4  104.96               83.84        23.0                 21.12   485.76

Total profit across dataset: 127941908.47999999


### 6) Save cleaned dataset

You can save the cleaned dataframe to a new CSV for further use.

In [21]:
cleaned_path = 'delhi_metro_cleaned_Dataset.csv'
df.to_csv(cleaned_path, index=False)
print('Saved cleaned CSV to', cleaned_path)

Saved cleaned CSV to delhi_metro_cleaned_Dataset.csv
